# SEQUENCE PREDICTION


## Goal

Instead of only pitches, we would like to include other aspects of music in our model.
These include:
* note lengths
* Rests
This is the only way to create rhythms that make the generated music more interesting.
We still limit ourselves to piano midi files, which can contain two tracks due to the notation of the two hands.

## Procedure

We want to keep the LSTM architecture, because LSTM is ideally suited for sequences.
Therefore we thought a lot about the music decoding.
Thus, the relevant information should be kept in as few parameters as possible.
To do this, we developed the following ideas:

### Music rasterization
A grid is placed over the music with a resolution greater than the measure (e.g., an eighth note). At each point in this grid, we consider whether a note is struck or held.
Thus, note lengths and rests are encoded.

* Advantages:
 * Output is one-dimensional
 * Low complexity

* Disadvantages:
 * Some notes are not considered in this grid because they are faster (e.g. 1/16 or triplets).
 * High complexity in pre-processing
 * Simultaneous notes have to be encoded/decoded as chords
 
This idea was not pursued further due to the high complexity in pre-processing.

### Temporal Difference (TD) focused approach

The Temporal Difference is the temporal distance between the attack of two successive notes.
For this purpose, each Global Offset of all notes in a piece of music was assigned the notes played at that time.
Thus, the two piano voices could be combined into one.
Since there can be notes of different length in the voices at one time, the length is stored together with each note.
An element of this TD-list is coded like this:

`[temporal_difference,[[pitch, duration], [pitch, duration], ...]]`

The LSTM must receive a uniform input. So the list of pitches must always be the same length. For this reason we have padded the list with empty elements up to a certain length.

The attempt to encode music in this form one-hot failed.
A vector of this type has a length of about 14,000, which is unsuitable for training.
Instead of hot encoding, TD lists could be output directly. However, the floats contained in them are not generated precisely enough and therefore cannot be mapped directly to the target values.

Experiments exist on this idea, but they are not presented in this notebook. The reduction of this approach provides the best result and is therefore part of this notebook.

### Reduction of the TD-focused approach.

The dimension problem of the previous approach is solved by splitting the TD lists.
Thus, the new input to the network is to consist of a sequence of lists of 3 elements each. Each of these lists contains the distance to the next note (TD), the duration as well as the pitch.
This way, multiple notes can still be played at the same time, since the TD is 0 in this case.

Advantages:
 * Dimensions are greatly reduced compared to the previous approach.
 * Several notes can be played at the same time and with different lengths
 * Different distances between notes are possible

#### Observations

* The parameter `sequence_length` must not be chosen too small. In an experiment with length 24 (v2), continuous loops of the same melodies or patterns occurred, since the training data also repeated at this length.
* A larger Sequence Length lengthens the training significantly.
* We expect a multidimensional output from the network, matching the input. For this reason, we have added another reshape layer to the architecture.
* The longer we train the mesh, the more it adapts to the training data. As a result, you get almost identical training data as output. So the mesh can easily overfit.

#### Results
* Subfolders results_RNN contain checkpoints of some tests with different parameters. The config.txt describes the configuration for the respective network. In addition, a notebook for the respective test is deposited. Since the training takes extremely long in each case, we could train only few variants.
* **Experiment V0**: The network is to be trained with only one piece of music. The goal is to test the basic functionality. By long training and a small dropout rate of 0.1 the loss shall be reduced to a minimum, which means strong overfitting. The output of the network is therefore very similar to the input.
* **Experiment V1**: The network was trained on several pieces of music by one composer. The architecture was slightly enlarged and the `sequence_length` was increased to 140.
* **Experiment V2**: The network was trained on several pieces of music by one composer. The `sequence_length` was significantly reduced with a value of 24. The result from this experiment sounds best. To avoid repetitive melodies the `sequence_length` can be increased for the generation. 
* **Experiment V3**: Trained with a mean `sequence_length` of 100 on several pieces of music by a different composer than in the previous trials. The number of parameters that can be learned is decreased.
* **Experiment V4**: In experiment v4, we put all the training songs in the same key. The expectation is that the generated music sounds more harmonic and the input space is reduced. Contrary to our expectation, we need much more time for training. The result has also deteriorated significantly.

In [1]:
################## Imports ##################
from music21 import converter, instrument, note, chord, stream, tempo
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, CuDNNLSTM, Reshape
from tqdm import tnrange
import numpy as np
import os
import glob

In [2]:
################## Pre Processing ##################

######### PARAMETERS #########
# preprocessing
music_directories = ('beeth', )
sequence_length = 140

# training
training_epochs = 100
training_batch_size = 1024

# inference
generated_notes_length = 300
file_name = 'test_gen_beeth.mid'

In [3]:
################## Pre Processing ##################
######### FUNCTIONS #########

# Load music file in given directories. Merge multiple hands and return music dictionary. This dictionary contains all notes for both hands at each global timestep
def load_music(data_dirs):
    music = {}
    total_offset = 0
    # load all files in all dirs
    for path in data_dirs:
        for file in glob.glob(path + '/*.mid'):
            print('parsing file ' + file)
            midi = converter.parse(file)
            notes_to_parse = midi.recurse()
            # falls mindestens 1 element in music, get totaloffset as biggest element of music
            if len(music.keys()) > 0:
                total_offset= sorted(music.keys())[-1]
            #go through all nodes
            for el in notes_to_parse:
                #check if offset is defined as float already 
                offset = float(el.offset)
                offset += total_offset
                #check if offset already occured before, no merge both the part of left and right hand
                if offset not in music:
                    music[offset] = []
                if isinstance(el, note.Note):
                    music[offset].append(el)
                elif isinstance(el, chord.Chord):
                    for element in el.notes:
                        music[offset].append(element)
                else:
                    if offset in music: del music[offset]
    return music

# If a given note (pitch) is flat, transform into sharp (requires flat_to_sharp mapping)
def to_sharp(pitch):
    # Mapping from flat to sharp notes in order to decrease dimensionality
    flat_to_sharp={
        'D-':'C#',
        'E-':'D#',
        'G-':'F#',
        'A-':'G#',
        'B-':'A#'
    }
    #pitch=e.g. 'E-4'
    if pitch not in ['-1']:
        if '-' in pitch:
            #flat!
            #get octave:
            octave = pitch[-1]
            return flat_to_sharp[pitch[:2]] + octave#0 to 1
        else:
            return pitch
    else:
        return pitch
    
#sort notes: 1. octave 2. pitch =>actual hight, not character
def sort_notes(unique_note_freqs):
#define all possible pitches
    possible_pitches = ['C','C#','D','D#','E','F','F#','G', 'G#','A','A#','B']
    all_pitches=[]
    for i in range(0,8):#1 to 7
        for pitch in possible_pitches:
            all_pitches.append(pitch+str(i))
    valid_pitches=[]
    for all_pitch in all_pitches:
        if all_pitch in unique_note_freqs:
            valid_pitches.append(all_pitch)
    return valid_pitches

# Convert music dictionary to music list
def music_dict_to_list(music_dict):
    music_list=[]
    for key in sorted(music_dict):
        music_list.append([key, music_dict[key]])
    return music_list

# get unqiue list of notes from given music list
def get_unique_notes(music_list):
    unique_notes = []
    for _, notes in music_list:
        #for each note      
        for _, current_note in enumerate(notes):
            current_pitch = to_sharp(str(current_note.pitch))
            #get all unique notes:
            if current_pitch not in unique_notes:
                unique_notes.append(current_pitch)
    return unique_notes

# get unique list of durations form given music list
def get_unique_durations(music_list):
    unique_durations = []
    for _, notes in music_list:
        #for each note      
        for _, current_note in enumerate(notes):
            current_duration = float(current_note.quarterLength)
            #get all unique durations:
            if current_duration not in unique_durations:
                unique_durations.append(current_duration)
    return unique_durations

# get unique list of durations form given music list
def get_unique_temporal_differences(music_list):
    unique_temporal_differences = [0.0]
    i = 0
    for _, notes in music_list:
        #define temporal difference
        #temporal difference: time until next note in list
        #e.g. if it's 0 it play simultaneously with next note
        #check if next note at i+1 exists to calculate time until next note
        if len(music_list) > i+1:
            #temporal differnce = duration from current timestamp to next timestamp
            temporal_difference = music_list[i+1][0] - music_list[i][0]
        else:
            #for last note
            #temporal differnce = duration of note
            temporal_difference = notes[0].quarterLength
        if temporal_difference not in unique_temporal_differences:
            unique_temporal_differences.append(temporal_difference)
        i+=1
    return unique_temporal_differences

# transform a music list into training data. We select 3 features for each note.
def create_training_data(music_list):
    training_data = []
    i=0
    for timestamp, notes in music_list:
        if len(music_list) > i+1:
            #temporal differnce = duration from current timestamp to next timestamp
            temporal_difference = music_list[i+1][0] - music_list[i][0]
        else:
            #for last note
            #temporal differnce = duration of note
            temporal_difference = notes[0].quarterLength
        #for each note      
        for idx, current_note in enumerate(notes):
            current_pitch = to_sharp(str(current_note.pitch))#to_sharp translates notes with 2 names => reduce input space
            current_duration = float(current_note.quarterLength)
            #append to training_data 
            if (idx+1) == len(notes):
                #last note in timestamp
                training_data.append([temporal_difference, current_pitch, current_duration])
            else:
                #temporal difference of 0 for simultaneous notes
                training_data.append([0, current_pitch, current_duration])
        i += 1
    return training_data

# create training sequences and the correpsonding outputs for given training data
def create_sequences(training_data, sequence_length):
    X = []
    y = []
    for i in range(0, len(training_data) - sequence_length):
        sequence_in = training_data[i:i + sequence_length]
        sequence_out = training_data[i + sequence_length]
        X.append(sequence_in)
        y.append(sequence_out)
    return X, y

# normalize input sequences
def normalize_input(X, unique_note_to_int, unique_duration_to_int, unique_temporal_difference_to_int):
    normalized_X=[]
    for sequence in X:
        normalized_sequence_X = []
        for data in sequence:
            normalized_single_X=[]
            # normalize temporal_diff:
            normalized_single_X.append(unique_temporal_difference_to_int[data[0]]/len(unique_temporal_difference_to_int.keys()))
            # normalize pitch
            normalized_single_X.append(unique_note_to_int[data[1]]/len(unique_note_to_int.keys()))
            # normalize duration
            normalized_single_X.append(unique_duration_to_int[data[2]]/len(unique_duration_to_int.keys()))
            # append normalized feature set to sequence
            normalized_sequence_X.append(normalized_single_X)
        normalized_X.append(normalized_sequence_X)
    return normalized_X

# normalize output feature set
def normalize_output(y, unique_note_to_int, unique_duration_to_int, unique_temporal_difference_to_int):
    normalized_y = []
    for data in y:
        normalized_single_y=[]
        # normalize temporal_diff:
        normalized_single_y.append(unique_temporal_difference_to_int[data[0]])
        # normalize pitch
        normalized_single_y.append(unique_note_to_int[data[1]])
        # normalize duration
        normalized_single_y.append(unique_duration_to_int[data[2]])
        # append normalized feature set to y
        normalized_y.append(normalized_single_y)
    normalized_y = to_categorical(normalized_y)
    return normalized_y

# predict output feature sets with trained model
def predict_notes(model, pattern, generated_notes_length, unique_temporal_differences_length, unique_notes_length, unique_durations_length):
    prediction_output = []
    for note_index in tnrange(generated_notes_length, desc='generating notes'):
        # predict next feature set with given pattern
        prediction_input = np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1]))
        
        #print("prediction input:\n",prediction_input)
        
        prediction = model.predict(prediction_input, verbose=0)
        
        # get max predictions
        index_temporal_difference = np.argmax(prediction[0][0])
        index_note = np.argmax(prediction[0][1])
        index_duration = np.argmax(prediction[0][2])
        
        #print("prediction 0", prediction[0][0])
        #print("prediction 1", prediction[0][1])
        #print("prediction 2", prediction[0][2])
        
        #print("prediction index 0", index_temporal_difference)
        #print("prediction index 1", index_note)
        #print("prediction index 2", index_duration)

        # transform prediction at max index
        result_temporal_difference = int_to_unique_temporal_difference[index_temporal_difference]
        result_note = int_to_unique_note[index_note]
        result_duration = int_to_unique_duration[index_duration]
        
        # Normalize predictions to append it to the sequence
        result_temporal_difference_normalized = index_temporal_difference/unique_temporal_differences_length
        result_note_normalized = index_note/unique_notes_length
        result_duration_normalized = index_duration/unique_durations_length
        
        #print("result 0", result_temporal_difference)
        #print("result 1", result_note)
        #print("result 2", result_duration)

        # append result to output
        prediction_output.append([result_temporal_difference, result_note, result_duration])
        
        # add new pattern in order to generate next feature set
        #pattern = np.vstack((pattern, (result_temporal_difference,result_note_normalized,result_duration)))
        pattern = np.vstack((pattern, (result_temporal_difference_normalized,result_note_normalized,result_duration_normalized)))
        pattern = np.delete(pattern, 0, 0)
    return prediction_output

# convert 
def convert_musiclist_to_music(prediction_output):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for feature_set in prediction_output:  
        # get single values from generated feature set
        gen_offset = feature_set[0]
        gen_note = feature_set[1]
        gen_duration = feature_set[2]

        # create note
        new_note = note.Note(gen_note, quarterLength=gen_duration)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

        # increase offset each iteration
        offset += gen_offset
    return output_notes

# play generated music
def play_music(output_notes, file_name):
    midi_stream = stream.Stream(output_notes) #output_notes
    #mm1 = tempo.MetronomeMark('slow')
    #midi_stream.append(mm1)
    #midi_stream.append(output_notes)
    midi_stream.write('midi', fp=file_name)
    midi_stream.show('midi')
    #midi_stream.show('text')

In [4]:
################## Pre Processing ##################
######### LOGIC #########

# load music files
music = load_music(data_dirs=music_directories)
print("# Parsed music length:",len(music))

# convert music dictionary to list
music_list = music_dict_to_list(music)

# get unique notes from music list
unique_notes = get_unique_notes(music_list)
#sort unique notes by actual pitch
unique_notes = sort_notes(unique_notes)
print("# Unique notes:\n", unique_notes)

# get unique durations from music list 
unique_durations = get_unique_durations(music_list)
print("# Unique durations:", unique_durations)
# get max duration from music list
max_duration = max(unique_durations)
print("# Max duration:", max_duration)

# get unqiue temporal difference
unique_temporal_differences = get_unique_temporal_differences(music_list)
print("# Unique temporal differences", unique_temporal_differences)
# get max temporal difference
max_temporal_difference = max(unique_temporal_differences)
print("# Max temporal difference:", max_temporal_difference)

# create training data by extracting 3 required features for each note
training_data = create_training_data(music_list)
print("# First element of training data (temporal_difference, note, duration):\n",training_data[0])

## create Mappings for all features
# create a dictionary to map notes to integers
unique_note_to_int = {note: number for number, note in enumerate(unique_notes)}
print("# Unique note to int:\n", unique_note_to_int)
int_to_unique_note = {number: note for number, note in enumerate(unique_notes)}
print("# Int to unique note:\n", int_to_unique_note)

# create a dictionary to map durations to integers
unique_duration_to_int = {duration: number for number, duration in enumerate(unique_durations)}
print("# Unique duration to int:\n", unique_duration_to_int)
int_to_unique_duration = {number: duration for number, duration in enumerate(unique_durations)}
print("# Int to unique duration:\n", int_to_unique_duration)

# create a dictionary to map temporal differences to integers
unique_temporal_difference_to_int = {temporal_difference: number for number, temporal_difference in enumerate(unique_temporal_differences)}
print("# Unique temporal difference to int:\n", unique_temporal_difference_to_int)
int_to_unique_temporal_difference = {number: temporal_difference for number, temporal_difference in enumerate(unique_temporal_differences)}
print("# Int to unique temporal difference:\n", int_to_unique_temporal_difference)

# create input sequences and the corresponding outputs
X, y = create_sequences(training_data, sequence_length)
print("# First X\n", X[0])
print("# First y\n:", y[0])

# normalize input and output
X = normalize_input(X, unique_note_to_int, unique_duration_to_int, unique_temporal_difference_to_int)
y = normalize_output(y, unique_note_to_int, unique_duration_to_int, unique_temporal_difference_to_int)
print("# First normalized X\n", X[0])
print("# First normalized y\n:", y[0])

# reshape the input into a format compatible with LSTM layers
X = np.reshape(X, (-1, sequence_length, 3))
print("# Shape of X (after reshape):", X.shape)
print("# Shape of y (after reshape):", y.shape)

parsing file beeth/mond_3.mid
parsing file beeth/beethoven_opus10_3.mid
parsing file beeth/beethoven_opus22_3.mid
parsing file beeth/pathetique_2.mid
parsing file beeth/waldstein_2.mid
parsing file beeth/beethoven_opus90_2.mid
parsing file beeth/beethoven_hammerklavier_4.mid
parsing file beeth/appass_2.mid
parsing file beeth/beethoven_hammerklavier_3.mid
parsing file beeth/waldstein_1.mid
parsing file beeth/beethoven_les_adieux_1.mid
parsing file beeth/appass_1.mid
parsing file beeth/beethoven_opus10_1.mid
parsing file beeth/beethoven_opus22_2.mid
parsing file beeth/beethoven_hammerklavier_1.mid
parsing file beeth/appass_3.mid
parsing file beeth/beethoven_opus90_1.mid
parsing file beeth/elise.mid
parsing file beeth/pathetique_1.mid
parsing file beeth/beethoven_hammerklavier_2.mid
parsing file beeth/beethoven_opus22_4.mid
parsing file beeth/pathetique_3.mid
parsing file beeth/mond_1.mid
parsing file beeth/beethoven_les_adieux_3.mid
parsing file beeth/beethoven_les_adieux_2.mid
parsing f

# First normalized X
 [[0.009523809523809525, 0.2564102564102564, 0.0], [0.0, 0.32051282051282054, 0.0], [0.009523809523809525, 0.2564102564102564, 0.0], [0.009523809523809525, 0.358974358974359, 0.0], [0.0, 0.41025641025641024, 0.0], [0.009523809523809525, 0.16666666666666666, 0.0], [0.009523809523809525, 0.32051282051282054, 0.0], [0.0, 0.358974358974359, 0.0], [0.009523809523809525, 0.2564102564102564, 0.0], [0.009523809523809525, 0.41025641025641024, 0.0], [0.0, 0.47435897435897434, 0.0], [0.009523809523809525, 0.16666666666666666, 0.0], [0.009523809523809525, 0.358974358974359, 0.0], [0.0, 0.41025641025641024, 0.0], [0.009523809523809525, 0.2564102564102564, 0.0], [0.009523809523809525, 0.47435897435897434, 0.0], [0.0, 0.5128205128205128, 0.0], [0.009523809523809525, 0.16666666666666666, 0.0], [0.009523809523809525, 0.41025641025641024, 0.0], [0.0, 0.47435897435897434, 0.0], [0.009523809523809525, 0.2564102564102564, 0.0], [0.009523809523809525, 0.5128205128205128, 0.0], [0.0, 0.5

In [5]:
################## Model ##################
model = Sequential()
model.add(CuDNNLSTM(
    1024,
    return_sequences=True, 
    input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(rate=0.3))
model.add(CuDNNLSTM(1024, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(CuDNNLSTM(512))
model.add(Dropout(rate=0.3))
model.add(Dense(units=y.shape[1] * y.shape[2], activation='softmax'))
model.add(Reshape((y.shape[1] ,y.shape[2])))
model.load_weights('results_RNN/v1/newchpts_beeth_950e')
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 140, 1024)         4214784   
_________________________________________________________________
dropout (Dropout)            (None, 140, 1024)         0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 140, 1024)         8396800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 140, 1024)         0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 512)               3149824   
_________________________________________________________________
dropout_2 (Dropout)  

In [6]:
################## Training ##################
# Wir laden bereits trainierte Gewichte, daher ist das trainig auskommentiert.
"""
checkpoint = ModelCheckpoint(
    'newchpts',
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

callbacks_list = [checkpoint]
model.fit(X, y, epochs=training_epochs, batch_size=training_batch_size, callbacks=callbacks_list)
"""

"\ncheckpoint = ModelCheckpoint(\n    'newchpts',\n    monitor='loss',\n    verbose=0,\n    save_best_only=True,\n    mode='min'\n)\n\ncallbacks_list = [checkpoint]\nmodel.fit(X, y, epochs=training_epochs, batch_size=training_batch_size, callbacks=callbacks_list)\n"

In [11]:
################## Inference ##################

# get random notes from training data
start = np.random.randint(0, len(X)-1)
seed = X[start]
# predict model output of specified length, starting with the previously created seed
prediction_output = predict_notes(model, seed, generated_notes_length, len(unique_temporal_differences), len(unique_notes), len(unique_durations)) 

# Convert feature set back to music
output_notes = convert_musiclist_to_music(prediction_output)

# Finally: Play music!
play_music(output_notes, file_name)

# Ideas

Ideas for the future ;)

## Separate models for note, duration and temporal difference.

Three models are learned. All models get the same input (sequences consisting of note, duration and temporal difference).
Each model provides the output for one of the 3 features. The individual features are then memorized for generation and can be played back as midi. Each model therefore focuses on the generation of a specific feature.

## Feed music separately

The individual music features probably do not fit together. It is very likely that the individual sequences overlap during training. Thus, the network learns with notes that do not match.
One idea to solve this problem is to train the net with the different pieces of music one after the other. A loop that trains the network for each file in a folder a few epochs at a time would be conceivable.

## Separate music
Another way to solve the problem described before would be to extend the music to separate them. This could be done by inserting long pauses after each track.